In [58]:
%load_ext autoreload
%autoreload 2

from functools import partial

from wordle.wordlenp import Wordle
from environment.environment import Environment, StateYesNo, ActionVocabulary
from dqn.agent import Agent
from dqn.train import train

import torch
import numpy as np
np.random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
n_answers = 10
n_guesses = 100

answers = Wordle._load_vocabulary('wordle/answers.txt', astype=np.array)
guesses = np.random.choice(answers, size=n_guesses, replace=False)
answers = np.random.choice(guesses, size=n_answers, replace=False)
answers

array(['scowl', 'swung', 'ditto', 'sinew', 'saucy', 'buxom', 'trice',
       'local', 'store', 'photo'], dtype='<U5')

In [63]:
# init environment
rewards = {'B':0, 'Y':1, 'G':1, 'win':10, 'lose':-10, 'step':-2}
env = Environment(
    rewards=rewards,
    wordle=Wordle(vocabulary=guesses, answers=answers),
    state_instance=StateYesNo())

action_constructor = partial(ActionVocabulary, vocabulary=guesses)

# init action
agent = Agent(
    state_size=env.state.size,
    action_size=len(env.wordle.vocabulary),
    action_constructor=action_constructor)

In [64]:
train(env=env, agent=agent)


Episode  100	Average Score: -14.28	Success Rate: 8.0%	Duration: 1.2

Episode  200	Average Score: -16.41	Success Rate: 3.0%	Duration: 2.6

Episode  300	Average Score: -13.35	Success Rate: 14.0%	Duration: 16.6

Episode  400	Average Score: -1.19	Success Rate: 61.0%	Duration: 28.6

Episode  500	Average Score: 0.18	Success Rate: 66.0%	Duration: 39.2

Episode  600	Average Score: 3.92	Success Rate: 79.0%	Duration: 48.3
sum_prob before 1.0177142119709792
sum_prob after :  0.9999999999999981

Episode  700	Average Score: 5.90	Success Rate: 88.0%	Duration: 57.5

Episode  800	Average Score: 5.78	Success Rate: 87.0%	Duration: 65.6

Episode  900	Average Score: 6.46	Success Rate: 90.0%	Duration: 74.4

Episode 1000	Average Score: 6.40	Success Rate: 89.0%	Duration: 83.2

Episode 1100	Average Score: 5.33	Success Rate: 86.0%	Duration: 93.1

Episode 1200	Average Score: 6.95	Success Rate: 91.0%	Duration: 101.9

Episode 1300	Average Score: 5.86	Success Rate: 87.0%	Duration: 110.9

Episode 1400	Average Scor

([-17,
  -15,
  7,
  -15,
  -14,
  -17,
  -15,
  -17,
  -16,
  -18,
  -16,
  6,
  -16,
  -16,
  -17,
  -15,
  -16,
  -17,
  -16,
  -13,
  -13,
  -15,
  -15,
  7,
  -15,
  -18,
  -17,
  -14,
  -19,
  -16,
  -18,
  5,
  -11,
  -17,
  -16,
  7,
  10,
  -15,
  -18,
  -16,
  -15,
  -17,
  -16,
  -17,
  -17,
  -16,
  -17,
  -15,
  -15,
  -18,
  -15,
  -17,
  -16,
  -17,
  -16,
  -16,
  -15,
  -15,
  -16,
  -15,
  -18,
  -17,
  -19,
  -14,
  -17,
  -14,
  -14,
  -16,
  -18,
  -16,
  -18,
  6,
  -14,
  -17,
  -16,
  10,
  -15,
  -16,
  -17,
  -13,
  -15,
  -17,
  -15,
  -17,
  -19,
  -16,
  -15,
  -15,
  -17,
  -19,
  -16,
  -17,
  -16,
  -19,
  -20,
  -15,
  -18,
  -16,
  -17,
  -18,
  -16,
  -17,
  -15,
  -17,
  -15,
  -19,
  -16,
  -18,
  -18,
  -15,
  -17,
  -15,
  -18,
  -19,
  -13,
  -17,
  -19,
  -17,
  -16,
  -18,
  8,
  -18,
  -19,
  -16,
  -19,
  -17,
  -17,
  -18,
  -18,
  7,
  -17,
  -18,
  -14,
  -15,
  -12,
  -17,
  -18,
  -19,
  -17,
  -18,
  -16,
  -11,
  -18,
  -18,
  -18,
  -